In [1]:
# Imports
from random import randint, sample, uniform, choice, choices
from operator import attrgetter
from copy import deepcopy
from tqdm import tqdm # Python library that produces a progress bar
import random

import numpy as np
from tensorflow.keras import Sequential, layers
import os
import csv

random.seed(1234)  # Ensuring reproducibility of the results

# Defining parameters for Image Classification problem
INPUT_SIZE = 28 * 28
HIDDEN_SIZE = 512
OUTPUT_SIZE = 10
ACTIVATION_1 = "relu"
ACTIVATION_2 = "softmax"
ACCURACY = 'categorical_accuracy'

In [2]:
from tensorflow.keras.datasets import mnist

# Load MNIST image datasets and labels
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape and normalize the input images
train_images = train_images.reshape(-1, 784) / 255.0
test_images = test_images.reshape(-1, 784) / 255.0

In [3]:
#######################################
## Individual and Population Classes ##
#######################################

class Individual: # Class defining an individual for the Genetic Algorithm
    # Each individual is characterized by a set of weights (representing a neural network's weights),
    # the number of input/hidden/output neurons (the neural network has 1 hidden layer)
    # and has a fitness score which is intended to be determined by the external function in mnist.py (get_fitness)
    def __init__( # Constructor method that gets called upon creation of a new instance of the class
            self,
            representation=None,
            input=None,
            hidden=None,
            output=None,
            valid_range=[]
    ):
        if representation is None: # If no representation is provided, generate random weights in the given valid_range
            size = input * hidden + hidden + hidden * output + output
            random_weights = [random.uniform(valid_range[0], valid_range[1]) for _ in range(size)]
            self.representation = {
                "weights": random_weights,
                "input": input,
                "hidden": hidden,
                "output": output
            }
        else: # If representation is provided, assign it directly
            self.representation = representation
        self.fitness = self.get_fitness()
        self.test_fitness = self.get_test_fitness()
        

    def get_fitness(self): # Fitness function evaluates the quality or performance of an individual solution
        """Method to calculate Fitness for MNIST image classification problem.

        Note:
            This function is intended to be monkey patched as the 'get_fitness' method into Individual class.
        Returns:
            float: Accuracy computed for a given instance of Individual class.
            In case of MNIST image classification, this function returns categorical_accuracy
            calculated on the training dataset (60,000 handwritten digits).
        """
        input = self.representation["input"]
        hidden = self.representation["hidden"]
        output = self.representation["output"]

        # Defining the architecture of a neural network
        nn = Sequential([
            layers.Dense(hidden, activation=ACTIVATION_1, input_shape=(input,)),
            layers.Dense(output, activation=ACTIVATION_2)
        ])

        current_weights = np.array(self.representation["weights"])
        weights_1 = current_weights[:input * hidden].reshape(input, hidden)
        biases_1 = current_weights[input * hidden:input * hidden + hidden].reshape(hidden, )
        weights_2 = current_weights[input * hidden + hidden:input * hidden + hidden + hidden * output].reshape(hidden, output)
        biases_2 = current_weights[input * hidden + hidden + hidden * output:].reshape(output, )

        # Setting the weights of the model
        nn.set_weights([weights_1, biases_1, weights_2, biases_2])

        # Compiling the model
        nn.compile(metrics=[ACCURACY])

        # Evaluating the model on training dataset
        train_loss, train_acc = nn.evaluate(train_images, train_labels, verbose=0)
        return train_acc

    def get_test_fitness(self): # Fitness function evaluates the quality or performance of an individual solution
        """Method to calculate Fitness for MNIST image classification problem.

        Note:
            This function is intended to be monkey patched as the 'get_fitness' method into Individual class.
        Returns:
            float: Accuracy computed for a given instance of Individual class.
            In case of MNIST image classification, this function returns categorical_accuracy
            calculated on the training dataset (60,000 handwritten digits).
        """
        input = self.representation["input"]
        hidden = self.representation["hidden"]
        output = self.representation["output"]

        # Defining the architecture of a neural network
        nn = Sequential([
            layers.Dense(hidden, activation=ACTIVATION_1, input_shape=(input,)),
            layers.Dense(output, activation=ACTIVATION_2)
        ])

        current_weights = np.array(self.representation["weights"])
        weights_1 = current_weights[:input * hidden].reshape(input, hidden)
        biases_1 = current_weights[input * hidden:input * hidden + hidden].reshape(hidden, )
        weights_2 = current_weights[input * hidden + hidden:input * hidden + hidden + hidden * output].reshape(hidden, output)
        biases_2 = current_weights[input * hidden + hidden + hidden * output:].reshape(output, )

        # Setting the weights of the model
        nn.set_weights([weights_1, biases_1, weights_2, biases_2])

        # Compiling the model
        nn.compile(metrics=[ACCURACY])

        # Evaluating the model on training dataset
        test_loss, test_acc = nn.evaluate(test_images, test_labels, verbose=0)
        return test_acc


    def index(self, value): # Returns the index of the provided value in the weights of the representation
        return self.representation["weights"].index(value)

    def __len__(self): # Returns the length of the weights in the representation
        return len(self.representation["weights"])

    def __getitem__(self, position): # Returns the value of the weight at the specified position
        return self.representation["weights"][position]

    def __setitem__(self, position, value): # Sets the weight at the specified position to the provided value
        self.representation["weights"][position] = value

    def __repr__(self): # Returns a string representation of the object, called whenever the object is printed
        # In this case, the returned string contains details about the individual and its fitness
        representation_print = str(self.representation["input"])+", " + str(self.representation["hidden"])+", " + str(self.representation["output"])
        return f"\nIndividual: {representation_print}; Fitness: {self.fitness}; Test_fitness: {self.test_fitness};"



class Population: # Class defining Population of individuals in Genetic Algorithm
    def __init__(self, size, optim, **kwargs):
        print("Generating initial population...")
        self.individuals = []
        self.size = size # initial population size is determined by the parameter size
        self.optim = optim # parameter to determine whether an optimization problem is maximization or minimization
        for _ in tqdm(range(size)):
            self.individuals.append(
                Individual( # **kwargs is used to pass the parameters to the Individual class to initialize each individual
                    input=kwargs["sol_input"],
                    hidden=kwargs["sol_hidden"],
                    output=kwargs["sol_output"],
                    valid_range=kwargs["valid_range"],
                )
            )

    def evolve(self, gens, select, crossover, mutate, xo_prob, mut_prob, elitism, **kwargs):
        """
        Method to evolve the population for a given number of generations,
        with a given selection method, crossover, mutation, and elitism.

        Args:
            gens (int): The number of generations to evolve the population for
            select (func): The selection method to use
            crossover (func): The crossover method to use
            mutate (func): The mutation method to use
            xo_prob (float): Crossover probability
            mut_prob (float): Mutation probability
            elitism (bool): Whether to use elitism
            **kwargs: Optionally, may contain 'tourn_size' (int), when selection is 'tournament_sel'.

        Returns:
            (list): The list of the best fitness values got for each generation during evolution.
            The length of the list is equal to 'gens' - number of generations.
        """

        best_fitness_lst = []  # list to store the best fitness for each generation
        test_fitness_lst = []

        for i in tqdm(range(gens)):
            # Printing out the current generation number each time a new generation starts
            print("Starting Generation ", i + 1, "...")
            new_pop = [] # at the beginning of each generation, new population is empty

            if elitism:
                if self.optim == "max":
                    elite = deepcopy(max(self.individuals, key=attrgetter("fitness")))
                elif self.optim == "min":
                    elite = deepcopy(min(self.individuals, key=attrgetter("fitness")))

            while len(new_pop) < self.size: # generate new individuals until the population size is reached
                # Selection
                if (select.__name__ == 'tournament_sel') and ('tourn_size' in kwargs):
                    parent1, parent2 = select(self, kwargs['tourn_size']), select(self, kwargs['tourn_size'])
                else:
                    parent1, parent2 = select(self), select(self)

                # Crossover
                if random.random() < xo_prob:
                    offspring1, offspring2 = crossover(parent1, parent2)
                else:
                    offspring1, offspring2 = parent1, parent2

                # Mutation
                if random.random() < mut_prob:
                    offspring1 = mutate(offspring1)
                if random.random() < mut_prob:
                    offspring2 = mutate(offspring2)

                # The new individuals are of the same architecture (input, hidden, and output) as their parents
                new_pop.append(Individual(representation={
                    "weights": offspring1,
                    "input": parent1.representation["input"],
                    "hidden": parent1.representation["hidden"],
                    "output": parent1.representation["output"],
                }))
                if len(new_pop) < self.size:
                    new_pop.append(Individual(representation={
                        "weights": offspring2,
                        "input": parent1.representation["input"],
                        "hidden": parent1.representation["hidden"],
                        "output": parent1.representation["output"],
                    }))

            # If elitism is enabled, the worst individual in the new population is replaced with
            # the elite individual from the old population.
            # This ensures that the best solution is not lost during evolution.
            if elitism:
                if self.optim == "max":
                    worst = min(new_pop, key=attrgetter("fitness"))
                    if elite.fitness > worst.fitness:
                        new_pop.pop(new_pop.index(worst))
                        new_pop.append(elite)

                elif self.optim == "min":
                    worst = max(new_pop, key=attrgetter("fitness"))
                    if elite.fitness < worst.fitness:
                        new_pop.pop(new_pop.index(worst))
                        new_pop.append(elite)

            # Replacing the population with the new one - completing the transition to the next generation
            self.individuals = new_pop

            # Finding the best individual in generation
            if self.optim == "max":
                best_individual = max(self, key=attrgetter("fitness"))
            elif self.optim == "min":
                best_individual = min(self, key=attrgetter("fitness"))

            # Printing out information about the best individual
            print(f'Best Individual in Generation {i + 1}: {best_individual}')

            # Appending the list of the best fitness values that will be returned once evolution ends
            best_fitness_lst.append(best_individual.fitness)
            test_fitness_lst.append(best_individual.fitness)

        return best_fitness_lst

    def __len__(self): # Returns the number of individuals in the population
        return len(self.individuals)

    def __getitem__(self, position): # Returns an individual at the given position in the population
        return self.individuals[position]


In [4]:
###############
## Selection ##
###############
def tournament_sel(population, tourn_size=4):
    """Tournament selection implementation.

    Args:
        population (Population): The population we want to select from.
        tourn_size (int): Size of the tournament.

    Returns:
        Individual: The best individual in the tournament.
    """
    # Select individuals based on tournament size
    # with choice, there is a possibility of repetition in the choices,
    # so every individual has a chance of getting selected
    tournament = [choice(population.individuals) for _ in range(tourn_size)]

    # with sample, there is no repetition of choices
    # tournament = sample(population.individuals, size)
    if population.optim == "max":
        return max(tournament, key=attrgetter("fitness"))
    if population.optim == "min":
        return min(tournament, key=attrgetter("fitness"))


In [5]:
###############
## Crossover ##
###############
def uniform_xo(p1, p2):
    """ Implementation of uniform crossover.
    Each gene in the parent, by random choice, decides if it will be swapped.
    This is done by generating a random binary mask of 1s and 0s, where a 1 indicates the gene will be swapped and a 0 - will not.
    Therefore, it is necessary to iterate over every gene and decide whether to swap it.

    Args:
        p1 (Individual): First parent for crossover.
        p2 (Individual): Second parent for crossover.

    Returns:
        Individuals: Two offspring, resulting from the crossover.
    """
    mask = choices([0, 1], k=len(p1))  # random binary mask
    offspring1, offspring2 = p1[:], p2[:]  # copies of parents
    for i in range(len(p1)):
        if mask[i] == 1:  # if the mask at position i is 1, swap the genes at position i
            offspring1[i], offspring2[i] = offspring2[i], offspring1[i]
    return offspring1, offspring2


In [6]:
##############
## Mutation ##
##############
def arithmetic_mutation(individual, lower_bound=-0.1, upper_bound=0.1):
    """Arithmetic mutation for a GA individual. Each gene is altered by a random value.

    Args:
        individual (Individual): A GA individual
        lower_bound (float): The lower bound for the random value
        upper_bound (float): The upper bound for the random value

    Returns:
        Individual: Mutated Individual
    """
    for i in range(len(individual)):
        # Generating and adding a random value to the gene
        mutation_value = random.uniform(lower_bound, upper_bound)
        individual[i] += mutation_value

        # Ensure the gene is within the desired range
        individual[i] = max(min(individual[i], 1.0), -1.0)

    return individual


In [7]:
def run_evolution(runs, pop_size, gens, select, crossover, mutate, xo_prob, mut_prob, elitism, output_file, **kwargs):
    """
    Function to perform a certain number of runs for the evolution algorithm of specific configuration.
    During the function execution, details about the best fitness values for each generation are printed out
    and progress bars are shown to track iterations.
    Upon each run completion, the results are appended into 'output_file'.

    Args:
        runs (int): Number of runs to perform evolution algorithm with the specified parameters
        pop_size (int): Population size
        gens (int): The number of generations to evolve the population for
        select (func): The selection method to use
        crossover (func): The crossover method to use
        mutate (func): The mutation method to use
        xo_prob (float): Crossover probability
        mut_prob (float): Mutation probability
        elitism (bool): Whether to use elitism
        output_file (str): File name to store the output results
        **kwargs: Optionally, may contain 'tourn_size' (int), when selection is 'tournament_sel'

    """

    for r in range(runs):
        print(f"RUN #{r + 1}")

        # Generating Initial Population
        pop = Population(
            size=pop_size,
            sol_input=INPUT_SIZE,
            sol_hidden=HIDDEN_SIZE,
            sol_output=OUTPUT_SIZE,
            valid_range=[-1, 1],
            optim="max")
        print(pop.individuals)

        # Running evolution iterations
        print('Evolving...')
        best_fitness_lst = pop.evolve(gens=gens,
                                      select=select, crossover=crossover, mutate= mutate,
                                      xo_prob=xo_prob, mut_prob=mut_prob,
                                      elitism=elitism, **kwargs)

        print(best_fitness_lst)

        # Merging configuration and results of the current run into the 'output_row' array
        output_row = [r+1, runs, pop_size, gens, select.__name__, crossover.__name__, mutate.__name__, xo_prob, mut_prob]
        if (select.__name__ == 'tournament_sel') and ('tourn_size' in kwargs):
            output_row.append(kwargs['tourn_size'])
        elif (select.__name__ == 'tournament_sel') and ('tourn_size' not in kwargs):
            output_row.append(4)
        else:
            output_row.append("")
        output_row.append(best_fitness_lst)

        # Storing results of the current run into the 'output_file'
        if not os.path.exists(output_file):  # File does not exist, create it and write headers
            with open(output_file, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["run_number", "total_runs", "pop_size", "gens", "select", "crossover", "mutate",
                                 "xo_prob", "mut_prob", "tourn_size", "best_fitness_lst"])
        # Append new row of data to the 'output_file'
        with open(output_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(output_row)

In [8]:
run_evolution(runs=1, pop_size=20, gens=20,
              select=tournament_sel,
              crossover=uniform_xo,
              mutate=arithmetic_mutation,
              xo_prob=0.9, mut_prob=0.3, elitism=True,
              output_file='testing.csv',
              tourn_size=2)

RUN #1
Generating initial population...


100%|██████████| 20/20 [01:40<00:00,  5.05s/it]


[
Individual: 784, 512, 10; Fitness: 0.6577500104904175; Test_fitness: 0.6553999781608582;, 
Individual: 784, 512, 10; Fitness: 0.01068333350121975; Test_fitness: 0.009499999694526196;, 
Individual: 784, 512, 10; Fitness: 0.06091666594147682; Test_fitness: 0.05420000106096268;, 
Individual: 784, 512, 10; Fitness: 0.2095833271741867; Test_fitness: 0.1932000070810318;, 
Individual: 784, 512, 10; Fitness: 0.002166666556149721; Test_fitness: 0.002300000051036477;, 
Individual: 784, 512, 10; Fitness: 0.07453333586454391; Test_fitness: 0.07900000363588333;, 
Individual: 784, 512, 10; Fitness: 0.00019999999494757503; Test_fitness: 0.0003000000142492354;, 
Individual: 784, 512, 10; Fitness: 0.04111666604876518; Test_fitness: 0.04089999943971634;, 
Individual: 784, 512, 10; Fitness: 0.020283333957195282; Test_fitness: 0.017899999395012856;, 
Individual: 784, 512, 10; Fitness: 0.4204833209514618; Test_fitness: 0.4374000132083893;, 
Individual: 784, 512, 10; Fitness: 0.04026666656136513; Test_fit

  0%|          | 0/20 [00:00<?, ?it/s]

Starting Generation  1 ...


...

Best Individual in Generation 18: 
Individual: 784, 512, 10; Fitness: 1.0; Test_fitness: 0.9998999834060669;
Starting Generation  19 ...


 95%|█████████▌| 19/20 [32:44<01:39, 99.06s/it] 

Best Individual in Generation 19: 
Individual: 784, 512, 10; Fitness: 1.0; Test_fitness: 0.9998999834060669;
Starting Generation  20 ...


100%|██████████| 20/20 [34:27<00:00, 103.39s/it]

Best Individual in Generation 20: 
Individual: 784, 512, 10; Fitness: 1.0; Test_fitness: 1.0;
[0.7801166772842407, 0.7801166772842407, 0.7801166772842407, 0.9191833138465881, 0.9749166369438171, 0.9749166369438171, 0.9909999966621399, 0.9909999966621399, 0.9986666440963745, 0.9987000226974487, 0.9997333288192749, 0.9997333288192749, 0.9998499751091003, 0.9999833106994629, 0.9999833106994629, 1.0, 1.0, 1.0, 1.0, 1.0]
